# Destiny

#### Importando pacotes e baixando banco de dados disponibilizado pela CVM

In [4]:
# Bibliotecas de Manipulação de Dados e Visualização
import pandas as pd

# Bibliotecas de Machine Learning e Estatísticas
import numpy as np
from imblearn.over_sampling import SMOTE
from lazypredict.Supervised import LazyClassifier
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#Biblioteca para salvar os resultados
import joblib


In [ ]:
# Instalando as bibliotecas 
#%pip install pandas 
#%pip install numpy
#%pip install imblearn
#%pip install lazypredict
#%pip install scikit-learn


In [5]:

pd.set_option("display.max_columns", None)

df1 = pd.read_csv('.\Banco_de_dados\IM_Classes_230626_semNP.csv')
df2 = pd.read_csv('.\Banco_de_dados\IM_230626_semNP.csv')

#### Eliminando numeros nulos e duplicados

In [6]:
df1 = df1.dropna().drop_duplicates()
df2 = df2.dropna().drop_duplicates()

In [7]:
df1.columns

Index(['SK_Documento', 'Classe_Serie', 'Numero_Cotistas', 'Quantidade_Cotas',
       'Valor_Cota', 'Rentabilidade', 'Valor_Total_Captado',
       'Quantidade_Cotas_Emitidas', 'Valor_Total_Resgates',
       'Quantidade_Cotas_Resgatadas', 'Valor_A_Pagar',
       'Quantidade_Cotas_A_Resgatar', 'Valor_Amortizado_Cota',
       'Valor_Total_Amortizacao', 'Desempenho_Esperado',
       'Desempenho_Realizado'],
      dtype='object')

#### Separação de Features

In [8]:
Colunas_1_Quero = ['SK_Documento', 'Numero_Cotistas']


In [9]:
Colunas_2_Quero = ['SK_Documento', 'CNPJ_Administrador', 'Nome_Administrador', 'Nome_Fundo',
                   'Fundo_Exclusivo', 'Ativo', 'Ativo_Carteira', 'Ativo_Direitos_Aquisicao', 
                   'Ativo_Direitos_Aquisicao_Creditos_Vencer_Adimplentes', 'Ativo_Direitos_Aquisicao_Creditos_Inadimplentes', 'Ativo_Direitos_Sem_Aquisicao',
                   'Ativo_Direitos_Sem_Aquisicao_Creditos_Vencer_Adimplentes', 
                   'Ativo_Direitos_Sem_Aquisicao_Creditos_Vencer_Inadimplentes', 
                   'Ativo_Valores_Mobiliarios', 'Patrimonio_Liquido', 'Liquidez_Ate_30_Dias', 
                   'Liquidez_Ate_60_Dias', 'Liquidez_Ate_90_Dias', 'Liquidez_Ate_180_Dias', 'Liquidez_Ate_360_Dias',
                   'Liquidez_Acima_360_Dias','Carteira', 'Ativo_Direitos_Aquisicao_Parcelas_Inadimplentes', 
                   'Ativo_Direitos_Aquisicao_Creditos_Inadimplentes','Ativo_Direitos_Aquisicao_Creditos_Vencer_Inadimplentes',"Carteira_Direitos_Aquisicao_Inadimplentes"
                   ]

In [10]:
todas_colunas2 = list(df2.columns)
for coluna in todas_colunas2:
    if coluna not in Colunas_2_Quero:
        df2 = df2.drop(coluna,axis=1)

In [11]:
todas_colunas1 = list(df1.columns)
for coluna in todas_colunas1:
    if coluna not in Colunas_1_Quero:
        df1 = df1.drop(coluna,axis=1)

Agrupando Fundos pelo SK_Documento

In [12]:
df1 = df1.groupby("SK_Documento").mean()

Fazendo mais um tratamento das features

In [13]:
df = pd.merge(df1,df2, on='SK_Documento', how="right")

In [14]:
df = df.drop("Nome_Administrador",axis=1)
df = df.drop("CNPJ_Administrador",axis=1)
df = df.drop("SK_Documento",axis=1)

#### Cálculo da taxa de inadimplência

In [15]:
df["Taxa_Inadimplencia"] = df["Carteira_Direitos_Aquisicao_Inadimplentes"]/df["Patrimonio_Liquido"]

#### Agrupando as features em prazos

In [16]:
df['Liquidez_curtoPrazo'] = df['Liquidez_Ate_30_Dias'] + df['Liquidez_Ate_60_Dias']
df = df.drop(['Liquidez_Ate_30_Dias','Liquidez_Ate_60_Dias'],axis=1)
df['Liquidez_medioPrazo'] = df['Liquidez_Ate_90_Dias'] + df['Liquidez_Ate_180_Dias']
df = df.drop(['Liquidez_Ate_90_Dias','Liquidez_Ate_180_Dias'],axis=1)
df['Liquidez_longoPrazo'] = df['Liquidez_Ate_360_Dias'] + df['Liquidez_Acima_360_Dias']
df = df.drop(['Liquidez_Ate_360_Dias','Liquidez_Acima_360_Dias'],axis=1)

In [17]:
df.shape

(6622, 21)

#### Transformando features categóricas em numéricas

In [18]:
df['Fundo_Exclusivo'].unique()

array(['Não', 'Sim'], dtype=object)

In [19]:
df['Fundo_Exclusivo'] = df.Fundo_Exclusivo.replace({'Não':0, 'Sim':1})


In [20]:
colunas_numericas = []

for coluna in df.columns:
        if pd.api.types.is_numeric_dtype(df[coluna].dtype):
            colunas_numericas.append(coluna)

df_numerico = df[colunas_numericas]

#### Normalizando as colunas

In [21]:
# Normalizando as colunas numéricas 
scaler = MinMaxScaler()
todas_colunas = list(df_numerico.columns)
df[todas_colunas] = scaler.fit_transform(df[todas_colunas])

#### Clusterização

Tratamento dos dados para clusterização

In [22]:
for index, linha in df.iterrows():
    if linha['Taxa_Inadimplencia'] < 0:
        df = df.drop(index, axis=0)

Fazendo o algorítimo para clusterização

In [23]:
dados = df['Taxa_Inadimplencia']

In [24]:
dados = np.array(dados)

In [25]:
dados.reshape(-1, 1)

array([[0.6274202],
       [0.6274202],
       [0.6274202],
       ...,
       [0.6274202],
       [0.6274202],
       [0.6274202]])

In [26]:
# 1. Tratar os Valores Ausentes
data = df[['Taxa_Inadimplencia']]
data = data.fillna(data.mean())  # Preenche os valores NaN com a média da coluna

# 2. Normalizar os Dados
scaler = StandardScaler()
data = scaler.fit_transform(data)

# 3. Aplicar o K-means
K = 2
kmeans = KMeans(n_clusters=K, random_state=42)
kmeans.fit(data)


KMeans(n_clusters=2, random_state=42)

In [27]:
labels = kmeans.labels_

In [28]:
df['Cluster'] = labels

#### Separando dados para treino e teste

In [29]:
df_treinamento = df.drop('Nome_Fundo', axis=1)
df_treinamento.dropna(inplace=True)

X = df_treinamento.copy()
X = X.drop("Cluster", axis=1)
X = pd.DataFrame(X)

Y = df_treinamento["Cluster"]
Y = Y.values.reshape(-1, 1)
Y = pd.DataFrame(Y)

Train_X, Test_X, Train_Y, Test_Y = train_test_split(X,Y, test_size=0.3)


#### Resolvendo os dados desbalanceados

In [31]:
smote = SMOTE(random_state = 32)
X_smote_res, Y_smote_res = smote.fit_resample(Train_X, Train_Y)

#### Utilizando Lazy predict para analisar os melhores modelos

In [32]:

clf = LazyClassifier()
models = clf.fit(Train_X, Test_X, Train_Y, Test_Y)

100%|██████████| 29/29 [00:05<00:00,  5.04it/s]

[LightGBM] [Info] Number of positive: 30, number of negative: 4557
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000467 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4845
[LightGBM] [Info] Number of data points in the train set: 4587, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.006540 -> initscore=-5.023222
[LightGBM] [Info] Start training from score -5.023222
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

#### Descobrindo overfitting

In [33]:
display(models)

(                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
 Model                                                                           
 AdaBoostClassifier                 1.00               1.00     1.00      1.00   
 BaggingClassifier                  1.00               1.00     1.00      1.00   
 XGBClassifier                      1.00               1.00     1.00      1.00   
 SVC                                1.00               1.00     1.00      1.00   
 SGDClassifier                      1.00               1.00     1.00      1.00   
 RidgeClassifierCV                  1.00               1.00     1.00      1.00   
 RidgeClassifier                    1.00               1.00     1.00      1.00   
 RandomForestClassifier             1.00               1.00     1.00      1.00   
 QuadraticDiscriminantAnalysis      1.00               1.00     1.00      1.00   
 Perceptron                         1.00               1.00     1.00      1.00   
 PassiveAggressi

In [34]:
dados = df['Taxa_Inadimplencia'].values


#### Random Forest Classifier

In [35]:
rf = RandomForestClassifier(n_estimators=100, random_state=0, oob_score=True, max_features=8)

rf.fit(X_smote_res, Y_smote_res)
pred_rf = rf.predict(Test_X)

In [36]:
pred_rf = pd.Series(pred_rf)

In [37]:
acc = accuracy_score(Test_Y, pred_rf)
print(f'Acurácia: {acc:.2f}')

f1 = f1_score(Test_Y, pred_rf, average='weighted')
print(f'f1_score {f1:.2f}')


Acurácia: 1.00
f1_score 1.00


In [38]:
joblib.dump(rf, 'resultado_modelo_rf.pkl')

['resultado_modelo_rf.pkl']

#### KNN

In [39]:
knn = KNeighborsRegressor(n_neighbors=49)

knn.fit(X_smote_res, Y_smote_res)
pred_knn = knn.predict(Test_X)


mse_knn = mean_squared_error(Test_Y, pred_knn)

In [40]:
joblib.dump(knn, 'resultado_modelo_knn.pkl')

['resultado_modelo_knn.pkl']

#### Analise fundos quebrados

##### Separação dos nomes dos fundos quebrados

In [41]:
df['Nome_Fundo'] = df2['Nome_Fundo']  
fundos_quebrados = ['FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS INDIGO BARTER',
                    'SB CRÉDITO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS NÃO PADRONIZADOS MULTISSETORIAL',
                    'CAPTALYS FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS NÃO PADRONIZADOS - MAIS LOTES',
                    'FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS BRAVA CHALLENGE',
                    'CREDIHOME FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS',
                    'SAFIRA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS NÃO PADRONIZADOS',
                    'RUBI FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS MULTISETORIAL',
                    'FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS BULLLA',
                    'LS INTERBANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS',
                    'MANGALARGA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS NÃO-PADRONIZADOS',
                    'TURQUESA - FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS',
                    'ÔNIX FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS NÃO PADRONIZADO',
                    'RAVENNA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS NÃO PADRONIZADOS',
                    ]

##### criação de tabela só com esses fundos

In [42]:
df_fq = df[df['Nome_Fundo'].isin(fundos_quebrados)]

In [43]:
df_fq["Valor_em_Risco"] = df_fq["Ativo_Direitos_Aquisicao_Creditos_Vencer_Inadimplentes"]/df_fq["Patrimonio_Liquido"]
df_fq1 = df_fq.loc[(df_fq.Valor_em_Risco > 0)]

##### clusterização desses fundos

In [44]:
df_fqt = df_fq[['Ativo', 'Valor_em_Risco']].copy()

for index, linha in df_fqt.iterrows():
    if linha['Valor_em_Risco'] == 0:
        df_fqt = df_fqt.drop(index, axis=0)

df_fqt.dropna(subset=['Ativo', 'Valor_em_Risco'], inplace=True)

In [45]:

if df_fqt.empty:
    print("O DataFrame está vazio. Verifique seus dados.")
else:

    dados = df_fqt[['Ativo', 'Valor_em_Risco']].values


    if len(dados) >= 2:  
        K = 4
        kmeans = KMeans(n_clusters=K, init='k-means++', random_state=42)
        kmeans.fit(dados)

        cluster_labels = kmeans.labels_
        centroids = kmeans.cluster_centers_

        print(f"Labels dos Clusters: {cluster_labels}")
        print(f"Centróides dos Clusters: {centroids}")
    else:
        print("Não há amostras suficientes para aplicar o K-Means.")

O DataFrame está vazio. Verifique seus dados.


In [46]:
dados = df_fqt[['Ativo', 'Valor_em_Risco']].values

K = 4
kmeans = KMeans(n_clusters=K, init='k-means++', random_state=42)
kmeans.fit(dados)

cluster_labels = kmeans.labels_
centroids = kmeans.cluster_centers_

ValueError: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required by KMeans.